# Fine Tuning Transformer for Text Classification

In [1]:
import json
import torch
import numpy as np
import pandas as pd
import transformers
import transformers
import torch.nn as nn
from sklearn import metrics
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score, matthews_corrcoef
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, RobertaTokenizer, RobertaModel

/opt/conda/lib/python3.8/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")


In [2]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

# Datos

In [ ]:
df = pd.read_csv('dataset_en_train_completed.csv')
df=df.iloc[:4000,:]
df.category.value_counts()

,count
category,
CRITICAL,2621
CONSPIRACY,1379


# Datos Balanceados (Aumento de Datos)

In [ ]:
df= pd.read_csv('dataset_en_train_completed.csv')
df= df.iloc[:4000]
#df

df_augmented= pd.read_csv('dataset_en_train_completed.csv')
df_augmented= df_augmented.iloc[4000:]
#df_augmented

In [ ]:
original_counts = df['category'].value_counts()
df_conspiracy = df[df['category'] == 'CONSPIRACY']
df_conspiracy_augmented = df_augmented[df_augmented['category'] == 'CONSPIRACY']
critical_count = original_counts.get('CRITICAL', 0)  # Asegurarnos que 'CRITICAL' existe
conspiracy_count = original_counts.get('CONSPIRACY', 0)


In [ ]:
df_conspiracy = df_augmented[df_augmented['category'] == 'CONSPIRACY']

# Seleccionar 1242 filas aleatorias para balancear el dataset 
df_conspiracy_sampled = df_conspiracy.sample(n=1242, random_state=42)
df_combined = pd.concat([df, df_conspiracy_sampled])

df_combined.reset_index(drop=True, inplace=True)
df=df_combined.copy()

# Generación de Prompts (emoción y moralidad reflejada en texto)

In [ ]:
def add_context(df):
    df['text'] =   df['text'] + '. The text reflects the emotion: ' + df['max_emotion'] + ' and the moral value: ' + df['max_moral']
    return df


df=add_context(df)
df.loc[0].text

'how the cia is directly involved in every aspect of the creation of the vaccine passports . groups created by the cia like palantir , mitre , oracle , and google , funded through the cias venture capital firm , in q tel , are are involved . every one of them are listed on the organizational member lists of the private companies in charge of the creation of all worldwide passports . full article . dailyveracity . com the shadowy cia data firms behind the creation of digital vaccine passport ids full video . bitchute . com video ufysjysoe donna. The text reflects the emotion: inspiration and the moral value: loyalty'

# Transformar etiqueta de categórica a numérica

Critical = 1
Conspirancy = 0

In [ ]:
df['class'] = df['category'].apply(lambda x: 1 if x == 'CRITICAL' else 0)

new_df = df[['text', 'class']].copy()
new_df

,text,class
0,how the cia is directly involved in every aspe...,0
1,elon musk admits i felt like i was dying after...,1
2,the uk gov. quietly published data confirming ...,1
3,"the global economic terror regime, which is lo...",0
4,confirmed pharma fags crashed into a covid vac...,1
...,...,...
5237,coming soon doctors for covid ethics fourth sy...,1
5238,the vaccine mandates as i said last year have ...,1
5239,Minority report in real life a group of social...,0
5240,This is the reason why the Cabal does not want...,0


In [ ]:
new_df.loc[0].text

'how the cia is directly involved in every aspect of the creation of the vaccine passports . groups created by the cia like palantir , mitre , oracle , and google , funded through the cias venture capital firm , in q tel , are are involved . every one of them are listed on the organizational member lists of the private companies in charge of the creation of all worldwide passports . full article . dailyveracity . com the shadowy cia data firms behind the creation of digital vaccine passport ids full video . bitchute . com video ufysjysoe donna. The text reflects the emotion: inspiration and the moral value: loyalty'

# Parámetros



In [ ]:
MAX_LEN = 512
TRAIN_BATCH_SIZE = 16
VALID_BATCH_SIZE = 16
EPOCHS = 3
LEARNING_RATE = 2e-5
tokenizer = AutoTokenizer.from_pretrained('roberta-base')
model = AutoModelForSequenceClassification.from_pretrained('roberta-base')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
class CustomDataset(Dataset):

    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.text = dataframe['text']
        self.targets = self.data['class']
        self.max_len = max_len

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = str(self.text[index])
        text = " ".join(text.split())

        # Tokenize the text
        inputs = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )

        ids = inputs['input_ids'].squeeze()
        mask = inputs['attention_mask'].squeeze()


        return {
            'ids': ids,
            'mask': mask,
            'targets': torch.tensor(self.targets[index], dtype=torch.float)
        }

# Modelos


## Modelo Roberta Base 

In [ ]:
class RoBERTaClass(torch.nn.Module):
    def __init__(self):
        super(RoBERTaClass, self).__init__()
        self.l1 = transformers.RobertaModel.from_pretrained('roberta-base')
        self.l2 = torch.nn.Dropout(0.3)
        self.l3 = torch.nn.Linear(768, 1)  

    def forward(self, ids, mask):
        _, output_1= self.l1(ids, attention_mask = mask, return_dict=False)
        output_2 = self.l2(output_1)
        output = self.l3(output_2).squeeze(1)
        return output

model = RoBERTaClass()
model.to(device)

## Modelo Roberta Base Modificado

In [ ]:
import torch
import torch.nn as nn
import transformers

class RoBERTaClass(torch.nn.Module):
    def __init__(self):
        super(RoBERTaClass, self).__init__()
        # Cargar el modelo preentrenado roberta-base
        self.l1 = transformers.RobertaModel.from_pretrained('roberta-base')

        # Añadir Multi-Head Attention
        self.multihead_attention = nn.MultiheadAttention(embed_dim=768, num_heads=8, dropout=0.3, batch_first=True)

        # Dropout para regularización
        self.l2 = nn.Dropout(0.3)

        # Capa lineal para clasificación binaria
        self.l3 = nn.Linear(768, 1) 

    def forward(self, ids, mask):
        output = self.l1(ids, attention_mask=mask, return_dict=True)
        hidden_states = output.last_hidden_state  # (batch_size, seq_len, hidden_dim)

        # Paso 2: Aplicar Multi-Head Attention
        attn_output, _ = self.multihead_attention(hidden_states, hidden_states, hidden_states, key_padding_mask=~mask.bool())

        # Paso 3: Tomar el embedding del token [CLS] (posición 0)
        cls_embedding = attn_output[:, 0, :]  # [CLS] después de atención [batch_size, hidden_dim]

        # Paso 4: Aplicar Dropout
        output_2 = self.l2(cls_embedding)

        # Paso 5: Pasar por la capa de clasificación
        output = self.l3(output_2).squeeze(1)

        return output

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = RoBERTaClass().to(device)


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def loss_fn(outputs, targets):
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)


In [ ]:
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

# Fine Tuning (Cross Validation)

In [ ]:
def train(epoch, model, train_loader):
    model.train()  
    for _, data in enumerate(train_loader):
        ids = data['ids'].to(device, dtype=torch.long)
        mask = data['mask'].to(device, dtype=torch.long)
        targets = data['targets'].to(device, dtype=torch.float)

        optimizer.zero_grad()  
        outputs = model(ids, mask)  
        loss = loss_fn(outputs, targets)  

        if _ % 5000 == 0:  
            print(f'Epoch: {epoch}, Loss: {loss.item()}')

        loss.backward()  
        optimizer.step() 

def validation(model, data_loader, device):
    model.eval()  
    fin_targets = []  
    fin_outputs = []  

    with torch.no_grad():  
        for data in data_loader:
            ids = data['ids'].to(device)
            mask = data['mask'].to(device)
            targets = data['targets'].to(device)

            outputs = model(ids, mask)  
            outputs = torch.sigmoid(outputs)

            fin_targets.extend(targets.cpu().detach().numpy().tolist())
            fin_outputs.extend(outputs.cpu().detach().numpy().tolist())

    return fin_outputs, fin_targets


def evaluate_metrics(outputs, targets):
    outputs = [1 if x > 0.5 else 0 for x in outputs]
    accuracy = accuracy_score(targets, outputs)
    precision, recall, f1, _ = precision_recall_fscore_support(targets, outputs, average='binary')
    mcc = matthews_corrcoef(targets, outputs)  # Calcular MCC
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'mcc': mcc
    }
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print(f"MCC: {mcc:.4f}")


def cross_validate_model(model, dataframe, tokenizer,title, epochs=3, batch_size=16, k_folds=5):
    #kf = KFold(n_splits=k_folds, shuffle=True, random_state=42)
    kf = StratifiedKFold(n_splits=k_folds, shuffle=True, random_state=42)

    metrics_list = []
    for fold, (train_idx, val_idx) in enumerate(kf.split(dataframe, dataframe['class'])):
        print(f"\nFold {fold + 1}/{k_folds}")
        train_df = dataframe.iloc[train_idx].reset_index(drop=True)
        val_df = dataframe.iloc[val_idx].reset_index(drop=True)

        train_set = CustomDataset(train_df, tokenizer, MAX_LEN)
        val_set = CustomDataset(val_df, tokenizer, MAX_LEN)

        train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
        val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False)

        for epoch in range(epochs):
            train(epoch, model, train_loader)  

        outputs, targets = validation(model, val_loader, device)

        fold_metrics=evaluate_metrics(outputs, targets)
        metrics_list.append(fold_metrics)

    metrics_df = pd.DataFrame(metrics_list)

    metrics_df.to_csv(f'metrics{title}.csv', index=False)

    print('Cross-validation complete')

In [ ]:

cross_validate_model(model, new_df, tokenizer, 'moral_emo_512_8_2e5_newrl_skfold_augmented',epochs=3, batch_size=TRAIN_BATCH_SIZE, k_folds=5)



Fold 1/5
Epoch: 0, Loss: 0.6991593837738037
Epoch: 1, Loss: 0.05224072188138962
Epoch: 2, Loss: 0.12664887309074402

Fold 2/5
Epoch: 0, Loss: 0.006059621460735798
Epoch: 1, Loss: 0.046710580587387085
Epoch: 2, Loss: 0.01651146076619625

Fold 3/5
Epoch: 0, Loss: 0.0011810350697487593
Epoch: 1, Loss: 0.010932333767414093
Epoch: 2, Loss: 0.001720610773190856

Fold 4/5
Epoch: 0, Loss: 0.26116979122161865
Epoch: 1, Loss: 0.0011455223429948092
Epoch: 2, Loss: 0.0010025594383478165

Fold 5/5
Epoch: 0, Loss: 0.0012970640091225505
Epoch: 1, Loss: 0.007878231815993786
Epoch: 2, Loss: 0.0003566271043382585
Cross-validation complete


# Guardar el modelo (opcional)

In [ ]:

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
MODEL_PATH = "/content/drive/MyDrive/MASTER/NLP/NLP_PAN/modelroberta.pth"

torch.save(model.state_dict(), MODEL_PATH)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Model and tokenizer saved for future inference.


# Test

In [ ]:
test_df = pd.read_csv("dataset_en_test_cleaned.csv")
test_df['class'] = test_df['category'].apply(lambda x: 1 if x == 'CRITICAL' else 0)
test_df = test_df[['text', 'class']].copy()
test_df

,text,class
0,elon now confirming what we ve been suspecting...,0
1,keeping the pressure on the police to uphold t...,0
2,safe effective the greatest lie ever told . th...,1
3,cdc report admits . million people in the usa ...,1
4,how to use health to acquire totalitarian cont...,0
...,...,...
995,john d. rockefeller wiped out natural cures to...,0
996,fact check biden white house falsely accuses d...,1
997,w onset acral hand lesions following mrna vacc...,1
998,we will fire unvaccinated workers cohen hadad ...,1


In [ ]:
model.eval()
def predict(text, model, tokenizer):
    inputs = tokenizer.encode_plus(
        text,
        None,
        add_special_tokens=True,
        max_length=MAX_LEN,
        padding='max_length',
        truncation=True,
        return_tensors='pt'
    )

    ids = inputs['input_ids'].to(device)
    mask = inputs['attention_mask'].to(device)

    with torch.no_grad():
        outputs = model(ids, mask)
        probabilities = torch.sigmoid(outputs).cpu().numpy()
        prediction = 1 if probabilities[0] >= 0.5 else 0

    return prediction, probabilities[0]


def test_and_evaluate(model, tokenizer, test_df, filename="test_results.json"):
    predictions = []
    probabilities = []

    # Iterate over the test data and generate predictions
    for index, row in test_df.iterrows():
        text = row['text']
        prediction, probability = predict(text, model, tokenizer)  # Assuming predict() function is defined
        predictions.append(prediction)
        probabilities.append(probability)

    # Add predictions and probabilities to the DataFrame
    test_df['predictions'] = predictions
    test_df['probabilities'] = probabilities

    # Calculate metrics
    mcc = matthews_corrcoef(test_df['class'], test_df['predictions'])

    # Generate classification report
    results = classification_report(
        test_df['class'],
        test_df['predictions'],
        target_names=['CONSPIRANCY', 'CRITICAL'],  # Specify labels for better readability
        digits=5,
        output_dict=True
    )



    # Save results and MCC to a JSON file
    output_data = {
        "mcc": mcc,
        "classification_report": results
    }

    with open(filename, "w") as f:
        json.dump(output_data, f, indent=4)

    print(f"Test results saved to {filename}")
    print(f"Test MCC = {mcc}")

# Example usage:
test_and_evaluate(model, tokenizer, test_df, filename="/content/drive/MyDrive/MASTER/NLP/test_evaluation_results_512_2e5_moral_emotions_robertanew_skfold_augmented.json")

Test results saved to /content/drive/MyDrive/MASTER/NLP/test_evaluation_results_512_2e5_moral_emotions_robertanew_skfold_augmented.json
Test MCC = 0.8036425977219992
